In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import trim, lower,concat_ws, collect_list

In [0]:
spark = SparkSession.builder \
    .appName("data_modelling") \
    .getOrCreate()

In [0]:
df_read = spark.read.format("jdbc") \
    .option("url", "jdbc:sqlserver://destination-landing-db.database.windows.net:1433;database=destination") \
    .option("user", 'adminuser@destination-landing-db') \
    .option("password", 'Test1234') \
    .option("query", f"""
        SELECT
            'online_retail_dm' as data_model,
            'invoice_meta_data' as table_name,
            query1.src_count,
            query2.trg_count,
            query1.src_count - query2.trg_count as difference_count,
            CURRENT_TIMESTAMP as time_recorded
        FROM
            (SELECT COUNT(InvoiceNo) as src_count
             FROM destination.online_retail
             WHERE InvoiceDate = (SELECT MAX(InvoiceDate) FROM destination.online_retail)) as query1
        JOIN
            (SELECT COUNT(InvoiceNumber) as trg_count
             FROM data_model.invoice_metadata
             WHERE InvoiceDate = (SELECT MAX(InvoiceDate) FROM data_model.invoice_metadata)) as query2
        ON 1=1
    """) \
    .load()


In [0]:
df_read.show()

+----------------+-----------------+---------+---------+----------------+--------------------+
|      data_model|       table_name|src_count|trg_count|difference_count|       time_recorded|
+----------------+-----------------+---------+---------+----------------+--------------------+
|online_retail_dm|invoice_meta_data|     1632|     1613|              19|2024-01-28 11:54:...|
+----------------+-----------------+---------+---------+----------------+--------------------+



In [0]:
df_read.write.format("jdbc").mode("append").option("url","jdbc:sqlserver://destination-landing-db.database.windows.net:1433;database=destination")\
    .option("dbtable","reconcilliation.records_sync")\
    .option("user","adminuser")\
    .option("password","Test1234")\
    .option("mode","append")\
    .save()

In [0]:
# query = f"""
#             select 'online_retail_dm' as data_model, 'invoice_meta_data' as table_name, query1.src_count, query2.trg_count, query1.src_count - query2.trg_count as difference_count,CURRENT_TIMESTAMP as time_recorded  
#             from  
#             (
#             select count(*) as src_count from destination.online_retail where InvoiceDate = (select max(InvoiceDate) from destination.online_retail)) as query1
#             join (
#               select count(*) as trg_count from data_model.invoice_metadata where InvoiceDate = (select max(InvoiceDate) from data_model.invoice_metadata)
#             )as query2
#             on 1=1
#             """